In [202]:
import pandas as pd
import numpy as np

In [203]:
import re

In [204]:
# 读入爬虫爬到的数据
pd_data = pd.read_json('./jobInfo.json')

In [230]:
# 看看数据格式
pd_data.head()

,company_address,company_industrial,company_name,company_nature,company_scale,company_webpage,demand_number,education_degree,feedback_rate,job_category,job_name,job_nature,job_url,publish_date,salary,work_experience,work_position,salary_max,salary_min,publish_time
0,北京市朝阳区西大望路63号传奇星广场下广场10号,互联网/电子商务,北京网营时代电子商务有限公司,民营,20,NaN,2人,大专,-1.0,订单处理员,数据专员,全职,http://jobs.zhaopin.com/CC486287921J0004331001...,2018-05-11,4000-8000,1-3,北京,8000,4000,13:19:01
1,山东省济南市高新区浪潮路1036号,IT服务(系统/数据/维护),浪潮集团有限公司,国企,10000,http://www.inspur.com,1人,大专,-1.0,软件工程师,高级大数据开发,全职,http://jobs.zhaopin.com/120143601286267.htm,2018-05-14,15001-20000,3-5,广州,20000,15001,15:01:19
2,北京市海淀区中关村大街15,互联网/电子商务,大锤资产,民营,100-499,NaN,1人,本科,-1.0,其他,商务总监,全职,http://jobs.zhaopin.com/CC571396932J0004212691...,2018-05-03,20001-30000,5-10,北京,30000,20001,10:19:40
3,北京亦庄经济技术开发区天华南街3号,医疗/护理/美容/保健/卫生服务,北京辉腾汇金国际母婴护理有限公司,股份制企业,20-99,NaN,2人,大专,-1.0,业务拓展经理/主管,渠道专员,全职,http://jobs.zhaopin.com/CC441451639J0003553961...,2018-05-13,6000-8000,0-99,北京,8000,6000,19:25:18
4,北京市东城区后永康胡同17号东雍创业谷D座202,IT服务(系统/数据/维护),北京特里尼斯石油技术股份有限公司,股份制企业,20-99,http://www.t-petro.com/,1人,本科,-1.0,出纳员,出纳,全职,http://jobs.zhaopin.com/293360815250124.htm,2018-04-24,4500-6500,1-3,北京,6500,4500,11:21:42


In [206]:
# 看看详细信息
pd_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5684 entries, 0 to 5683
Data columns (total 18 columns):
company_address       5683 non-null object
company_industrial    5606 non-null object
company_name          5682 non-null object
company_nature        5682 non-null object
company_scale         5682 non-null object
company_webpage       2989 non-null object
demand_number         5683 non-null object
education_degree      5683 non-null object
feedback_rate         1548 non-null object
job_category          5683 non-null object
job_name              5683 non-null object
job_nature            5683 non-null object
job_url               5683 non-null object
publish_date          5683 non-null object
salary                5683 non-null object
scrape_time           5683 non-null object
work_experience       5683 non-null object
work_position         5683 non-null object
dtypes: object(18)
memory usage: 843.7+ KB


In [207]:
# 由于爬取的时候最后一行是空的字典，所以把最后一行删除
# 非常重要的一步! 不然会报各种莫名其妙的错误。
pd_data =  pd_data.loc[:5682,:]

In [208]:
# 用于削去公司地址中的 \n \r和空格等等
def parse_address(s):
    s = re.split(r'\\n|\r\\n|-|\s+',s)[1]
    return s

In [209]:
pd_data.loc[:,'company_address'] = pd_data['company_address'].apply(parse_address)

In [210]:
# 将反馈率归一化：没有填写反馈率的统一标记为 -1 , 其他反馈率在 [ 0,1 ]之间
def parse_rate(s):
    if s is not None:
        rate = int(re.split(r'%',s)[0])/100
    else:
        rate = -1
    return rate

In [211]:
pd_data.loc[:,'feedback_rate'] = pd_data['feedback_rate'].apply(parse_rate)

In [212]:
# 把数据中的“人”字剔除，把剩余部分转换成 int 或 str 方便后续喂给模型。
def parse_demand(d_num):
    number = int(re.split(r'人',d_num)[0])
    return number

def parse_scale(s_num):
    if s_num is not None:
        return str(re.split(r'人',s_num)[0])
    else:
        return 'UnKnown'

In [213]:
pd_data.loc[:,'company_scale'] = pd_data['company_scale'].apply(parse_scale)

In [214]:
# 同样把数据处理
def parse_work_exp(period):
    period = re.split(r'\s+',period)[0]
    if period is not None:
        if period == '不限':
            return '0-99'
        else:
            return str(re.split(r'年',period)[0])
    else:
        return 'Unknown'

In [215]:
pd_data.loc[:,'work_experience'] = pd_data['work_experience'].apply(parse_work_exp)

In [216]:
# 返回最大工资
def parse_salary_max(s):
    if s == '面议\xa0':
        return -1
    else:
        s = re.split(r'元/月',s)[0]
        return int(re.split(r'-',s)[-1])

#  返回最小工资
def parse_salary_min(s):
    if s == '面议\xa0':
        return -1
    else:
        s = re.split(r'元/月',s)[0]
        return int(re.split(r'-',s)[0])

In [217]:
pd_data.loc[:,'salary_max'] = pd_data['salary'].apply(parse_salary_max)
pd_data.loc[:,'salary_min'] = pd_data['salary'].apply(parse_salary_min)

In [218]:
# 将日期和时间分开
def parse_date(d):
    return d[:10]

def parse_time(t):
    return t[11:]

In [219]:
pd_data.loc[:,'publish_time'] =pd_data['publish_date'].apply(parse_time)
pd_data.loc[:,'publish_date'] =pd_data['publish_date'].apply(parse_date)

In [220]:
# 把没用的信息给剔除(比如爬取时间)
pd_data.pop('scrape_time')

0       2018_05_14_15_55_43
1       2018_05_14_15_55_45
2       2018_05_14_15_55_47
3       2018_05_14_15_55_48
4       2018_05_14_15_55_50
5       2018_05_14_15_55_51
6       2018_05_14_15_55_54
7       2018_05_14_15_55_56
8       2018_05_14_15_55_58
9       2018_05_14_15_56_00
10      2018_05_14_15_56_03
11      2018_05_14_15_56_05
12      2018_05_14_15_56_07
13      2018_05_14_15_56_09
14      2018_05_14_15_56_11
15      2018_05_14_15_56_13
16      2018_05_14_15_56_16
17      2018_05_14_15_56_18
18      2018_05_14_15_56_20
19      2018_05_14_15_56_23
20      2018_05_14_15_56_24
21      2018_05_14_15_56_26
22      2018_05_14_15_56_29
23      2018_05_14_15_56_30
24      2018_05_14_15_56_32
25      2018_05_14_15_56_34
26      2018_05_14_15_56_36
27      2018_05_14_15_56_38
28      2018_05_14_15_56_40
29      2018_05_14_15_56_43
               ...         
5653    2018_05_14_19_43_24
5654    2018_05_14_19_43_26
5655    2018_05_14_19_43_28
5656    2018_05_14_19_43_31
5657    2018_05_14_1

In [225]:
# 处理工资数据
def parse_salary(s):
    if s == '面议\xa0':
        return 'Unknown'
    else:
        s = re.split(r'元/月',s)[0]
        return s

In [226]:
pd_data.loc[:,'salary'] = pd_data['salary'].apply(parse_salary)

In [227]:
# 可以看到我们现在的数据就非常整齐了。
pd_data.head(7)

,company_address,company_industrial,company_name,company_nature,company_scale,company_webpage,demand_number,education_degree,feedback_rate,job_category,job_name,job_nature,job_url,publish_date,salary,work_experience,work_position,salary_max,salary_min,publish_time
0,北京市朝阳区西大望路63号传奇星广场下广场10号,互联网/电子商务,北京网营时代电子商务有限公司,民营,20,NaN,2人,大专,-1.0,订单处理员,数据专员,全职,http://jobs.zhaopin.com/CC486287921J0004331001...,2018-05-11,4000-8000,1-3,北京,8000,4000,13:19:01
1,山东省济南市高新区浪潮路1036号,IT服务(系统/数据/维护),浪潮集团有限公司,国企,10000,http://www.inspur.com,1人,大专,-1.0,软件工程师,高级大数据开发,全职,http://jobs.zhaopin.com/120143601286267.htm,2018-05-14,15001-20000,3-5,广州,20000,15001,15:01:19
2,北京市海淀区中关村大街15,互联网/电子商务,大锤资产,民营,100-499,NaN,1人,本科,-1.0,其他,商务总监,全职,http://jobs.zhaopin.com/CC571396932J0004212691...,2018-05-03,20001-30000,5-10,北京,30000,20001,10:19:40
3,北京亦庄经济技术开发区天华南街3号,医疗/护理/美容/保健/卫生服务,北京辉腾汇金国际母婴护理有限公司,股份制企业,20-99,NaN,2人,大专,-1.0,业务拓展经理/主管,渠道专员,全职,http://jobs.zhaopin.com/CC441451639J0003553961...,2018-05-13,6000-8000,0-99,北京,8000,6000,19:25:18
4,北京市东城区后永康胡同17号东雍创业谷D座202,IT服务(系统/数据/维护),北京特里尼斯石油技术股份有限公司,股份制企业,20-99,http://www.t-petro.com/,1人,本科,-1.0,出纳员,出纳,全职,http://jobs.zhaopin.com/293360815250124.htm,2018-04-24,4500-6500,1-3,北京,6500,4500,11:21:42
5,北京市东城区夕照寺街东玖大厦B座706室,媒体/出版/影视/文化传播,北京智博宝通传媒股份有限公司,上市公司,20-99,www.zibocn.com,1人,本科,-1.0,高级软件工程师,高级JAVA工程师,全职,http://jobs.zhaopin.com/149552214250285.htm,2018-05-10,20000-25000,5-10,北京,25000,20000,13:47:18
6,北京市东城区夕照寺街东玖大厦B座706室,媒体/出版/影视/文化传播,北京智博宝通传媒股份有限公司,上市公司,20-99,www.zibocn.com,1人,大专,-1.0,运营主管/专员,商城运营（有提成）,全职,http://jobs.zhaopin.com/149552214250271.htm,2018-05-03,3500-6000,1-3,北京,6000,3500,10:59:11


In [229]:
# 全部处理完之后，输出。
pd_data.to_json('./jobInfo_May.json')